In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import math
from scipy.stats import weibull_min
import random

In [2]:
step_size=[0.05,0.025,0.0125,0.0125]
REVERSALS=3
CONTRAST_VALUE= 0.25

In [3]:
responses_df = pd.read_csv("Data/Locations_mask.csv")
#simulated_threshold=[]
for i in range(len(responses_df)):
    #simulated_threshold.append(0.0)
    try:
        responses_df.at[i,"Nearest_prev_group_loc"] = [int(x) for x in (responses_df.iloc[i]["Nearest_prev_group_loc"].split(","))]   
    except:
        pass


responses_df.insert(5,"Contrast_list",value=[[] for i in range(len(responses_df))])
responses_df.insert(6,"Responses",value=[[] for i in range(len(responses_df))])
responses_df.insert(7,"Reversals",value=[0]*len(responses_df))
responses_df.insert(9,"Estimated Weibull Threshold",value=[0.0]*len(responses_df))
responses_df.insert(10,"True Weibull Slope",value=[2.5]*len(responses_df))

responses_df.head()

,Location,X,Y,Group,Nearest_prev_group_loc,Contrast_list,Responses,Reversals,True Weibull Threshold,Estimated Weibull Threshold,True Weibull Slope
0,0,2,7,3,"[4, 5, 6]",[],[],0,0.3,0.0,2.5
1,1,3,7,3,"[5, 6, 7]",[],[],0,0.4,0.0,2.5
2,2,4,7,3,"[6, 7, 8]",[],[],0,0.3,0.0,2.5
3,3,5,7,3,"[7, 8, 9]",[],[],0,0.3,0.0,2.5
4,4,1,6,2,[12],[],[],0,0.3,0.0,2.5


In [4]:
# reimplemented from matlab
# g=0
# required for fractional stepsize

def drange(start, stop, step):
    r = start
    while r < stop:
        yield r
        r += step

def weibull(x,k,l,g):
    # k = slope
    # l = threshold
    return g + (1 - g)*(1 - math.exp(-(x/l)**k))

def own_fitweibull(g=0):
    #location g = 0

    i0 = drange(-3,0,0.001)
    lmbda = [float(x) for x in i0]          #Scale parameter

    i1 = drange(2,2.5,0.1)
    k0 = [float(x) for x in i1]                  #Shape parameter or slope

    g=0

    for i in range(len(responses_df)):
        LL0 = float('-inf') 
        optLambda = float('-inf')
        mleK = float('-inf')

        contrast_list = responses_df.iloc[i]["Contrast_list"]
        responses = responses_df.iloc[i]["Responses"]

        df = pd.DataFrame(list(zip(contrast_list,responses)),columns=["Contrast","Response"])

        #convert responses to
        correct_responses = list(df.loc[df.Response=="Yes"].Contrast)
        incorrect_responses = list(df.loc[df.Response=="No"].Contrast)

        for cc in range(len(lmbda)):
            for ss in range(len(k0)):
                # log likelihood
                CR = []
                IR = []
                for response in correct_responses:
                    w_result = weibull(10**math.log10(response), k0[ss], 10**lmbda[cc], g);    # Correct responses
                    w_result = math.log(w_result)
                    #print(response,w_result)
                    CR.append(w_result)

                for response in incorrect_responses:
                    w_result = weibull(10**math.log10(response), k0[ss], 10**lmbda[cc], g);    # Incorrect responses
                    #print(w_result)
                    w_result = 1.01 - w_result          #see matlab implementation 1.01 because log0 is not defined
                    w_result = math.log(w_result)       #see matlab implementation
                    IR.append(w_result)

                LL = sum(CR) + sum(IR);                 #likelihood estimate

                if(LL>LL0):
                    optLambda = lmbda[cc]
                    mleK = k0[ss]
                    LL0 = LL
                    #print(LL0)

        # keep the most optimal values for threshold and slope in the matrix
        threshold = 10**optLambda
        slope = mleK
        responses_df.at[i,"Estimated Weibull Threshold"] = threshold
        #responses_df.at[i,"Weibull Slope"] = slope
    

In [5]:
def get_random_location(groupnumber):
    x = list(responses_df.loc[(responses_df["Group"]==groupnumber) & (responses_df["Reversals"]==0)].index)
    random.shuffle(x)
    return x

def count_reversals(contrast_list,response_list):
    count = 0
    prev_response=None
    
    for i in range(len(response_list)-1):
        if(response_list[i]!=response_list[i+1]):
            count+=1
            
    #if the contrast value is at the upper and lower limit consecutively then count it as reversal
    for j in range(len(contrast_list)-1):
        
        if(contrast_list[j]==1.0 and contrast_list[j+1]==1.0):
            if(response_list[j]==response_list[j+1]):
                count+=1
        if(contrast_list[j]==0.0 and contrast_list[j+1]==0.0):
            if(response_list[j]==response_list[j+1]):
                count+=1

    return count

# simplify and put everything in a single function

def get_nearest_ones_value(location):
    """
    Find the nearest 1s for the given location and obtain the contrast level to start
    """
    
    prev_group_location = responses_df.iloc[location]["Nearest_prev_group_loc"]

    try:
        #Estimated
        estimated_contrast_value = responses_df.iloc[prev_group_location[0]]["Contrast_list"][-1]
    except:
        estimated_contrast_value = CONTRAST_VALUE
        
    return estimated_contrast_value

def get_nearest_neighbours_value(location):
    """
    Find the nearest neighbours for the given location and obtain the AVERAGE contrast level to start
    """
    prev_group_location = responses_df.iloc[location]["Nearest_prev_group_loc"]
    
    try:
        contrast_list = []
        for i in range(len(prev_group_location)):
            last_contrast_level = responses_df.iloc[prev_group_location[i]]["Contrast_list"][-1]
            contrast_list.append(last_contrast_level)
            
        avg_contrast_value = sum(contrast_list)/len(contrast_list)
        
    except:
        print("Exception raise: location:",location)
        avg_contrast_value=CONTRAST_VALUE
        
    return avg_contrast_value

In [6]:
def get_new_location(random_location_list):
    not_done_list=[]
    for x in random_location_list:
        if(responses_df.iloc[x]["Reversals"]<3):
            not_done_list.append(x)
    
    if(len(not_done_list)!=0):
        location = random.choice(not_done_list)
    else:
        location = random_location_list[0]
    
    return location

In [7]:
def perform_simulated_responses(groupnumber):
    random_location_list = get_random_location(groupnumber)
    #print(random_location_list)
    location = random.choice(random_location_list)


    while responses_df.iloc[location]["Reversals"]<REVERSALS:

        #print("Starting trial for location:",location)

        contrast_list = responses_df.iloc[location]["Contrast_list"]
        responses = responses_df.iloc[location]["Responses"]

        #if the contrast list is empty(i.e there was no testing done before then initialise with a start value)
        if(len(contrast_list)==0):
            if(groupnumber==1):
                contrast_list=[CONTRAST_VALUE]
            if(groupnumber==2):
                estimated_contrast_value = get_nearest_ones_value(location)
                contrast_list = [estimated_contrast_value]
            if(groupnumber==3 or groupnumber == 4):
                estimated_contrast_value = get_nearest_neighbours_value(location)
                contrast_list = [estimated_contrast_value]

            probability_true = weibull(contrast_list[-1],k=responses_df.iloc[location]["True Weibull Slope"],
                                       l=responses_df.iloc[location]["Estimated Weibull Threshold"],g=0)
            output = np.random.choice([True,False],p=[probability_true,1-probability_true])
            if(output==True):
                responses.append("Yes")
            else:
                responses.append("No")

            responses_df.at[location,"Contrast_list"] = contrast_list #[1.0]
            responses_df.at[location,"Responses"] = responses #["yes"]

        # When there are already responses and contrast_list populated
        # contrast_list = [1.0]
        # repsonses = ["Yes"]
        else:

            #check previous response and update contrast list +-stepsize
            if(responses[-1]=="Yes"):
                contrast = contrast_list[-1] - step_size[responses_df.iloc[location]["Reversals"]]
                if(contrast<=0):
                    contrast = 0.001

            else:            
                contrast = contrast_list[-1] + step_size[responses_df.iloc[location]["Reversals"]]
                if(contrast>=1.0):
                    contrast = 1.0
            #print(contrast)
            contrast = round(contrast,5)
            contrast_list.append(contrast)

            #get new output for the new contrast_level
            probability_true = weibull(contrast_list[-1],k=responses_df.iloc[location]["True Weibull Slope"],
                           l=responses_df.iloc[location]["Estimated Weibull Threshold"],g=0)
            #print(probability_true)
            output = np.random.choice([True,False],p=[probability_true,1-probability_true])


            if(output==True):
                responses.append("Yes")
            else:
                responses.append("No")

            responses_df.at[location,"Contrast_list"] = contrast_list #[1.0,0.9]
            responses_df.at[location,"Responses"] = responses #["Yes","No"]

            #update number of reversals
            responses_df.at[location,"Reversals"] = count_reversals(contrast_list,responses)

        #randomise location within the group
        #pick new location within list whose reversals is less than 3
        location = get_new_location(random_location_list)

In [8]:
"""def perform_simulated_responses(groupnumber):
    
    #Given a group number it will perform the staircase method and collect responses until 3 reversals are recorded
    
    
    random_location_list = get_random_location(groupnumber)
    print(random_location_list)
    for location in random_location_list:

        #if group number is 1 then contrast elvel starts with 1.0
        if(groupnumber==1):
            contrast_list=[CONTRAST_VALUE]

        #else start with the average estimate of the nearests previous group
        if(groupnumber==2):
            estimated_contrast_value = get_nearest_ones_value(location)
            contrast_list = [estimated_contrast_value]

        if(groupnumber==3 or groupnumber == 4):
            estimated_contrast_value = get_nearest_neighbours_value(location)
            contrast_list = [estimated_contrast_value]

        responses=[]
        while(responses_df.iloc[location]["Reversals"]<REVERSALS):

            output = np.random.choice([True,False],p=[0.6,0.4]) #CHECK WITH CHRIS

            #print(output)

            if(output==True):
                responses.append("Yes")
                if(len(responses)>=2 and responses[-1]!=responses[-2]):
                    responses_df.at[location,"Reversals"] = responses_df.iloc[location]["Reversals"]+1
                
                contrast = contrast_list[-1]-step_size[responses_df.iloc[location]["Reversals"]]
                if(contrast<=0.1):
                    contrast=0.1
                contrast_list.append(contrast)
                    

            elif(output==False):
                responses.append("No")
                if(len(responses)>=2 and responses[-1]!=responses[-2]):
                    responses_df.at[location,"Reversals"] = responses_df.iloc[location]["Reversals"]+1
                   
                contrast = contrast_list[-1]+step_size[responses_df.iloc[location]["Reversals"]]
                if(contrast>=1.0):
                    #incase contrast level is upper limit then count as reversal+1
                    contrast=1.0
                    responses_df.at[location,"Reversals"] = responses_df.iloc[location]["Reversals"]+1
                    
                    
                contrast_list.append(contrast)#step_size/(responses_df.iloc[location]["Reversals"]+1))


        if(len(responses)!=len(contrast_list)):
            contrast_list=contrast_list[:-1]

        contrast_list = [round(elem, 2) for elem in contrast_list ]    
        responses_df.at[location,"Contrast_list"] = contrast_list
        responses_df.at[location,"Responses"] = responses"""

'def perform_simulated_responses(groupnumber):\n    \n    #Given a group number it will perform the staircase method and collect responses until 3 reversals are recorded\n    \n    \n    random_location_list = get_random_location(groupnumber)\n    print(random_location_list)\n    for location in random_location_list:\n\n        #if group number is 1 then contrast elvel starts with 1.0\n        if(groupnumber==1):\n            contrast_list=[CONTRAST_VALUE]\n\n        #else start with the average estimate of the nearests previous group\n        if(groupnumber==2):\n            estimated_contrast_value = get_nearest_ones_value(location)\n            contrast_list = [estimated_contrast_value]\n\n        if(groupnumber==3 or groupnumber == 4):\n            estimated_contrast_value = get_nearest_neighbours_value(location)\n            contrast_list = [estimated_contrast_value]\n\n        responses=[]\n        while(responses_df.iloc[location]["Reversals"]<REVERSALS):\n\n            output =

In [9]:
def simulation_method():
    #sample a random location
    #put random response
    #based on response do staircase procedure
    
    #start with group 1
    perform_simulated_responses(groupnumber=1)
    
    #group2
    perform_simulated_responses(groupnumber=2)
    
    #group3
    perform_simulated_responses(groupnumber=3)
    
    #group4
    perform_simulated_responses(groupnumber=4)
    

In [10]:
responses_df

,Location,X,Y,Group,Nearest_prev_group_loc,Contrast_list,Responses,Reversals,True Weibull Threshold,Estimated Weibull Threshold,True Weibull Slope
0,0,2,7,3,"[4, 5, 6]",[],[],0,0.30,0.0,2.5
1,1,3,7,3,"[5, 6, 7]",[],[],0,0.40,0.0,2.5
2,2,4,7,3,"[6, 7, 8]",[],[],0,0.30,0.0,2.5
3,3,5,7,3,"[7, 8, 9]",[],[],0,0.30,0.0,2.5
4,4,1,6,2,[12],[],[],0,0.30,0.0,2.5
5,5,2,6,2,[12],[],[],0,0.20,0.0,2.5
6,6,3,6,2,[12],[],[],0,0.10,0.0,2.5
7,7,4,6,2,[15],[],[],0,0.10,0.0,2.5
8,8,5,6,2,[15],[],[],0,0.10,0.0,2.5
9,9,6,6,2,[15],[],[],0,0.15,0.0,2.5


In [13]:
responses_df.iloc[39]

Location                                                                      39
X                                                                              5
Y                                                                              2
Group                                                                          1
Nearest_prev_group_loc                                                       NaN
Contrast_list                  [0.25, 0.2, 0.15, 0.1, 0.05, 0.001, 0.001, 0.0...
Responses                      [Yes, Yes, Yes, Yes, Yes, Yes, Yes, Yes, Yes, ...
Reversals                                                                      0
True Weibull Threshold                                                      0.05
Estimated Weibull Threshold                                                  0.0
True Weibull Slope                                                           2.5
Name: 39, dtype: object

In [12]:
simulation_method()

C:\Users\Bhara\AppData\Local\Temp\ipykernel_23572\491070624.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  return g + (1 - g)*(1 - math.exp(-(x/l)**k))

KeyboardInterrupt



In [ ]:
responses_df

In [ ]:
def plot_observations_loc(location):
    
    print("location:",location)
    
    observed_levels = responses_df.iloc[location]["Contrast_list"] # y axis

    plt.plot(range(len(observed_levels)),observed_levels,marker="D")
    # naming the x axis
    plt.xlabel('Trial number')
    # naming the y axis
    plt.ylabel('Contrast levels')
    plt.show()

In [ ]:
for i in range(len(responses_df)):
    plot_observations_loc(i)

- Staircase procedure just estimates the threshold of the contrast but does not help in estimating the slope of a pyschometric function
- We fit our data on a weibull distribution using a "maximum likelihood" procedure to pick levels that predict the correct performance

#### Weibull distribution
y = 1 - e^(x/lambda)^k

where 
    x = stimulus intensity
    y = percentage of correct responses
    lambda and k are free parameters which we estimate
    
#refer http://courses.washington.edu/matlab1/Lesson_5.html

our formula:
out = g + (1 - g).*(1 - exp(-(x./lambda).^k));


##### parameter search: 
 - scale parameter: lambda(-3:0,0.0001)
 - shape parameter: k0(2:2.5,0.1)
<br>These are taken from previous matlab code

In [ ]:
own_fitweibull()

In [ ]:
responses_df

In [ ]:
def plot_weibull(location):
    
    x0 = drange(0.001,1,0.001)
    xx = [float(x) for x in x0]
    
    l = responses_df.iloc[location]["Weibull Threshold"]
    k = responses_df.iloc[location]["Weibull Slope"]
    
    yy = [weibull(i,k,l,0) for i in xx]

    plt.plot(xx,yy)
    # naming the plot
    title = 'location:',location
    plt.title(title)
    plt.show()

In [ ]:
for i in range(len(responses_df)):
    plot_weibull(i)

#### Plot the resulting final contrast levels (last "Yes")